# 경활(EAPS) 청년 취업 예측 모델 (정리본)
**CSV 로드 → 전처리/파생변수 → 모델 학습(로지스틱/랜덤포레스트) → 평가/점수화** 흐름으로 정리했습니다.
- Colab 기준 예시이며, 로컬에서도 `BASE_DIR`만 바꾸면 동작합니다.


In [4]:
# =========================
# 0) 환경 설정 / 라이브러리
# =========================
import os, glob, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# (선택) 한글 폰트/마이너스 깨짐 방지 - Windows/Colab 환경에 따라 조정
plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["axes.unicode_minus"] = False

In [5]:
# =========================
# 1) 데이터 경로 설정
# =========================
# Colab에서 Google Drive 사용 시:
try:
    from google.colab import drive  # type: ignore
    drive.mount("/content/drive")
    BASE_DIR = "/content/drive/MyDrive/csv"
except Exception:
    # 로컬/Jupyter 환경이면 여기를 수정하세요.
    BASE_DIR = "../../data/eaps_year/"

print("BASE_DIR:", BASE_DIR)
print("예시 파일:", glob.glob(os.path.join(BASE_DIR, "*.csv"))[:5])

BASE_DIR: ../../data/eaps_year/
예시 파일: ['../../data/eaps_year\\eaps_year_2021.csv', '../../data/eaps_year\\eaps_year_2022.csv', '../../data/eaps_year\\eaps_year_2023.csv', '../../data/eaps_year\\eaps_year_2024.csv', '../../data/eaps_year\\eaps_year_2025.csv']


In [6]:
# =========================
# 2) 연도별 CSV 로드 & 결합
# =========================
def extract_year_from_filename(path: str):
    '''
    파일명에서 4자리 연도(예: 2019)를 찾습니다.
    - 못 찾으면 None 반환
    '''
    m = re.search(r"(19\d{2}|20\d{2})", os.path.basename(path))
    return int(m.group(1)) if m else None

csv_files = sorted(glob.glob(os.path.join(BASE_DIR, "*.csv")))
if not csv_files:
    raise FileNotFoundError(f"CSV 파일이 없습니다. BASE_DIR={BASE_DIR} 확인")

df_list = []
for f in csv_files:
    year = extract_year_from_filename(f)
    df = pd.read_csv(f, low_memory=False)
    if year is not None and "조사년도" not in df.columns:
        df["조사년도"] = year
    df_list.append(df)

df_eaps = pd.concat(df_list, ignore_index=True)
print("rows:", len(df_eaps), "cols:", df_eaps.shape[1])
df_eaps.head()

rows: 3409715 cols: 62


,동부읍면부코드,가구주관계코드,성별코드,출생연도,교육정도_학력구분코드,교육정도_계열코드,교육정도_수학구분코드,졸업연도,혼인상태코드,현재일관련사항_지난주수입목적근로여부,...,이전직장_종사상지위코드,이전직장_1자리_7차직업대분류코드,조사연월,농가비농가가구구분코드,교육정도컨버젼코드,만연령,연령5세단위코드,가중값,경제활동상태코드,조사년도
0,1,1,1,1923,2,0,1,0,3,2,...,0,0.0,202112,2,2,98,12,1234872,3,2021
1,1,1,1,1923,3,3,1,0,3,2,...,0,0.0,202101,2,3,97,12,359767,3,2021
2,1,1,1,1923,3,3,1,0,3,2,...,0,0.0,202102,2,3,97,12,356766,3,2021
3,1,1,1,1923,3,3,1,0,3,2,...,0,0.0,202103,2,3,97,12,363033,3,2021
4,1,1,1,1924,0,0,0,0,2,2,...,0,0.0,202105,2,1,97,12,299531,3,2021


In [7]:
# =========================
# 3) 타깃(y) 생성: 취업여부 (0/1)
# =========================
TARGET_SRC_COL = "현재일관련사항_지난주수입목적근로여부"

if TARGET_SRC_COL not in df_eaps.columns:
    raise KeyError(f"타깃 생성 컬럼이 없습니다: {TARGET_SRC_COL}")

# 예: 1=예, 2=아니오 같은 코드라면 '==1'로 0/1 변환
df_eaps["취업여부"] = (df_eaps[TARGET_SRC_COL] == 1).astype(int)

print(df_eaps["취업여부"].value_counts(dropna=False))

취업여부
1    1936187
0    1473528
Name: count, dtype: int64


In [8]:
# =========================
# 4) 전처리/파생 변수
# =========================
# 만나이 = 조사년도 - 출생연도
for col in ["조사년도", "출생연도"]:
    if col not in df_eaps.columns:
        raise KeyError(f"필수 컬럼 없음: {col}")

df_eaps["만나이"] = df_eaps["조사년도"] - df_eaps["출생연도"]

# 청년 범위(예시: 20~39세)
df_youth = df_eaps[(df_eaps["만나이"] >= 20) & (df_eaps["만나이"] <= 39)].copy()
print("전체:", len(df_eaps), "청년:", len(df_youth))

# 성별 더미(예: 1=남, 2=여)
if "성별코드" in df_youth.columns:
    df_youth["남성"] = (df_youth["성별코드"] == 1).astype(int)

# 학력 더미(예시: 5,6이 대졸 이상)
if "교육정도_학력구분코드" in df_youth.columns:
    df_youth["대졸이상"] = df_youth["교육정도_학력구분코드"].isin([5, 6]).astype(int)

# 이공계 더미(예시 코드 - 필요시 수정)
STEM_CODES = [10, 20, 21, 22, 30, 40, 50, 60, 70]
if "교육정도_계열코드" in df_youth.columns:
    df_youth["이공계"] = df_youth["교육정도_계열코드"].isin(STEM_CODES).astype(int)

df_youth[["만나이", "취업여부"] + [c for c in ["남성","대졸이상","이공계"] if c in df_youth.columns]].head()

전체: 3409715 청년: 731926


,만나이,취업여부,남성,대졸이상,이공계
159790,39,1,1,0,0
159791,39,1,1,0,0
159792,39,1,1,0,0
159793,39,1,1,0,0
159794,39,1,1,0,0


In [9]:
# =========================
# 5) 학습용 변수 선택 (FEATURES)
# =========================
FEATURES = [c for c in [
    "만나이", "남성", "대졸이상", "이공계",
    "혼인상태코드", "교육정도_학력구분코드", "교육정도_계열코드"
] if c in df_youth.columns]

if not FEATURES:
    raise ValueError("FEATURES가 비어있습니다. 컬럼명을 확인하세요.")

X = df_youth[FEATURES].copy()
y = df_youth["취업여부"].astype(int).copy()

# 결측 처리(안정적으로: 수치는 중앙값, 범주는 최빈값)
for c in FEATURES:
    if X[c].dtype == "O":
        X[c] = X[c].fillna(X[c].mode(dropna=True).iloc[0] if X[c].notna().any() else "MISSING")
    else:
        X[c] = X[c].fillna(X[c].median() if X[c].notna().any() else 0)

print("FEATURES:", FEATURES)
X.head()

FEATURES: ['만나이', '남성', '대졸이상', '이공계', '혼인상태코드', '교육정도_학력구분코드', '교육정도_계열코드']


,만나이,남성,대졸이상,이공계,혼인상태코드,교육정도_학력구분코드,교육정도_계열코드
159790,39,1,0,0,2,2,0
159791,39,1,0,0,2,2,0
159792,39,1,0,0,2,2,0
159793,39,1,0,0,2,2,0
159794,39,1,0,0,2,2,0


In [ ]:
# =========================
# 6) Train/Test split
# =========================
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("train:", X_train.shape, "test:", X_test.shape)
print("y_train ratio:", y_train.mean(), "y_test ratio:", y_test.mean())

In [ ]:
# =========================
# 7) 모델 1: 로지스틱 회귀 (Baseline)
# =========================
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

logit = Pipeline([
    ("scaler", StandardScaler(with_mean=True, with_std=True)),
    ("lr", LogisticRegression(max_iter=5000))
])

logit.fit(X_train, y_train)

def eval_binary(model, Xtr, ytr, Xte, yte, name="model"):
    p_tr = model.predict_proba(Xtr)[:, 1]
    p_te = model.predict_proba(Xte)[:, 1]
    pred_tr = (p_tr >= 0.5).astype(int)
    pred_te = (p_te >= 0.5).astype(int)

    return {
        "model": name,
        "train_acc": accuracy_score(ytr, pred_tr),
        "test_acc": accuracy_score(yte, pred_te),
        "train_auc": roc_auc_score(ytr, p_tr),
        "test_auc": roc_auc_score(yte, p_te),
    }

logit_res = eval_binary(logit, X_train, y_train, X_test, y_test, "LogisticRegression")
logit_res

In [ ]:
# (선택) 로지스틱 회귀 계수 확인 (표준화된 입력 기준)
lr = logit.named_steps["lr"]
coef = pd.Series(lr.coef_[0], index=FEATURES).sort_values()
coef.plot(kind="barh")
plt.title("Logistic Regression Coefficients")
plt.show()

In [ ]:
# =========================
# 8) 모델 2: 랜덤 포레스트 (주력)
# =========================
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=500,
    random_state=42,
    n_jobs=-1,
    class_weight="balanced_subsample"
)

rf.fit(X_train, y_train)

rf_res = eval_binary(rf, X_train, y_train, X_test, y_test, "RandomForest")
rf_res

In [ ]:
# (선택) 랜덤포레스트 중요도
imp = pd.Series(rf.feature_importances_, index=FEATURES).sort_values()
imp.plot(kind="barh")
plt.title("Random Forest Feature Importances")
plt.show()

In [ ]:
# =========================
# 9) Train/Test 성능 비교 + Overfitting 체크
# =========================
results = pd.DataFrame([logit_res, rf_res]).set_index("model")
results["acc_gap(train-test)"] = results["train_acc"] - results["test_acc"]
results["auc_gap(train-test)"] = results["train_auc"] - results["test_auc"]
results

In [ ]:
# =========================
# 10) 취업가능성 점수 산출 (RF 기준)
# =========================
df_score = df_youth.copy()
df_score["취업가능성점수"] = rf.predict_proba(X)[:, 1]

df_score["취업가능성점수"].describe()

In [ ]:
# (선택) 점수 분포 시각화
df_score["취업가능성점수"].plot(kind="hist", bins=30)
plt.title("취업가능성점수 분포 (RF)")
plt.xlabel("score")
plt.show()

In [ ]:
# =========================
# 11) 위험군/비위험군(예: 하위 20% vs 상위 20%) 비교 (선택)
# =========================
q_low = df_score["취업가능성점수"].quantile(0.2)
q_high = df_score["취업가능성점수"].quantile(0.8)

df_score["그룹"] = np.where(
    df_score["취업가능성점수"] <= q_low, "위험군(하위20%)",
    np.where(df_score["취업가능성점수"] >= q_high, "비위험군(상위20%)", "중간")
)

compare_cols = [c for c in [
    "만나이","남성","대졸이상","이공계",
    "혼인상태코드","교육정도_학력구분코드","교육정도_계열코드"
] if c in df_score.columns]

compare = (
    df_score[df_score["그룹"].isin(["위험군(하위20%)","비위험군(상위20%)"])]
    .groupby("그룹")[compare_cols].mean().T
)
compare["차이(비위험-위험)"] = compare["비위험군(상위20%)"] - compare["위험군(하위20%)"]
compare.sort_values("차이(비위험-위험)")

In [ ]:
# (선택) 차이 시각화
compare["차이(비위험-위험)"].sort_values().plot(kind="barh")
plt.title("위험군 vs 비위험군 평균 차이 (비위험-위험)")
plt.show()